In [ ]:
# Custom Params
IMAGE_SIZE = (128, 128)
TEST_DATA_SIZE = 0.2

# Data Loading

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Path to the dataset
data_dir = '/kaggle/input/plantdisease/PlantVillage'

# Classes (labels) based on folder names in the dataset
classes = [plant for plant in os.listdir("/kaggle/input/plantdisease/PlantVillage") if plant.startswith("Tomato")]
print(classes)

# Mapper dictionary
class_mapper = {
    'Tomato_Leaf_Mold': 'Leaf Mold',
    'Tomato__Tomato_YellowLeaf__Curl_Virus': 'Yellow Leaf Curl Virus',
    'Tomato_Bacterial_spot': 'Bacterial Spot',
    'Tomato_Septoria_leaf_spot': 'Septoria Leaf Spot',
    'Tomato_healthy': 'Healthy',
    'Tomato_Spider_mites_Two_spotted_spider_mite': 'Spider Mites',
    'Tomato_Early_blight': 'Early Blight',
    'Tomato__Target_Spot': 'Target Spot',
    'Tomato_Late_blight': 'Late Blight',
    'Tomato__Tomato_mosaic_virus': 'Mosaic Virus'
}

In [ ]:
X = []
y = []

for label, class_name in tqdm(enumerate(classes)):
    class_dir = os.path.join(data_dir, class_name)
    label_name = class_mapper[class_name]
    
    if not os.path.isdir(class_dir):
        continue
    for img_file in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_file)
        
        # Read the image using OpenCV
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, IMAGE_SIZE)           # Resize image: Preprocessing
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Converting BGR to RGB: Preprocessing
            X.append(img)
            y.append(label_name)

# Converting lists to numpy arrays
X = np.array(X)
y = np.array(y)

print(f"Data loaded successfully: {X.shape[0]} images with size {X.shape[1:]}")

# Data Exploration

In [ ]:
import matplotlib.pyplot as plt
from collections import defaultdict

samples_per_class = 5
fig, axes = plt.subplots(len(class_mapper), samples_per_class, figsize=(12, len(class_mapper) * 2))
fig.suptitle("Sample Images from Each Class", fontsize=16, weight='bold', y=1.02)

# Indices by class for easy access
class_indices = defaultdict(list)
for idx, label in enumerate(y):
    class_indices[label].append(idx)

# Display images
for class_idx, (class_name, indices) in enumerate(class_indices.items()):
    selected_indices = np.random.choice(indices, samples_per_class, replace=False)

    for i, idx in enumerate(selected_indices):
        ax = axes[class_idx, i]
        ax.imshow(X[idx])
        ax.axis("off")

    # Label as a title for each row
    axes[class_idx, 0].set_title(class_name, fontsize=12, loc='left', pad=5)

# Adjusting layout for better spacing
plt.subplots_adjust(wspace=0.1, hspace=0.4)
plt.tight_layout(rect=[0.05, 0.05, 1, 0.96])
plt.show()

In [ ]:
import seaborn as sns
from collections import Counter

# Occurrences of each class in y
class_counts = Counter(y)

# Extracting class names and counts for plotting
classes = list(class_counts.keys())
counts = list(class_counts.values())

# Bar plot
plt.figure(figsize=(12, 6))
sns.barplot(x=classes, y=counts, palette="viridis")

# Add titles and labels
plt.title("Distribution of Tomato Leaf Disease Classes", fontsize=16, weight='bold')
plt.xlabel("Class", fontsize=12)
plt.ylabel("Number of Images", fontsize=12)
plt.xticks(rotation=45, ha='right')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# Encoding y
y_encoded = [classes.index(i) for i in y]

# Spliting data into 80% training and 20% testing: Preprocessing
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, 
                                                    test_size=TEST_DATA_SIZE, 
                                                    stratify=y, random_state=42)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ImageDataGenerator with Augmentation for Training
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,              # Normalize images: Preprocessing
    
    rotation_range=20,              # Rotate images up to 20 degrees
    width_shift_range=0.1,          # Shift images horizontally by 10%
    height_shift_range=0.1,         # Shift images vertically by 10%
    shear_range=0.2,                # Shear transformation
    zoom_range=0.2,                 # Zoom in on images
    horizontal_flip=True,           # Randomly flip images horizontally
    fill_mode='nearest'             # Fill pixels created by transformation
)

# Generator for test data (only rescaling, no augmentation)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Training generator
train_generator = train_datagen.flow(
    X_train, y_train,
    batch_size=BATCH_SIZE,        # Grouping: Preprocessing
    shuffle=True                  # Shuffle the data
)

# Testing generator
test_generator = test_datagen.flow(
    X_test, y_test,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Summary
print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")


# CNN Model

import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(DROPOUT_RATIO),
    Dense(len(classes), activation='softmax')  # Output layer with units equal to the number of classes
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # Use sparse_categorical_crossentropy for integer labels
    metrics=['accuracy']
)